In [1]:
%env MINIO_HOST=minio
%env MINIO_PORT=9000
%env AWS_SECRET_KEY=EI1RW90H9KpLAaG8AOJlK1rnsEnWhfM5gvcfqq9d
%env AWS_ACCESS_KEY=SFFTcTnl3QKhg5XxVW6O

env: MINIO_HOST=minio
env: MINIO_PORT=9000
env: AWS_SECRET_KEY=EI1RW90H9KpLAaG8AOJlK1rnsEnWhfM5gvcfqq9d
env: AWS_ACCESS_KEY=SFFTcTnl3QKhg5XxVW6O


In [2]:
import os
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,io.delta:delta-spark_2.12:3.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executorEnv.AWS_ACCESS_KEY", os.environ["AWS_ACCESS_KEY"]) \
    .config("spark.executorEnv.AWS_SECRET_KEY", os.environ["AWS_SECRET_KEY"]) \
    .config("spark.hadoop.fs.s3a.endpoint", f"http://{os.environ['MINIO_HOST']}:{os.environ['MINIO_PORT']}") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

In [3]:
import random

data = [(random.randint(1, 100), random.randint(1, 100)) for _ in range(100)]
df = spark.createDataFrame(data, ["col1", "col2"])
df.write.format("csv").mode("overwrite").save("s3a://landing/test-csv-table")

In [4]:
data = [(random.randint(1, 100), random.randint(1, 100)) for _ in range(100)]
df = spark.createDataFrame(data, ["col1", "col2"])
df.write.format("delta").mode("overwrite").save("s3a://landing/test-delta-table")